<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/drum_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skorch
!pip install tqdm

     |████████████████████████████████| 133kB 30.9MB/s 


# Plot Utils 

In [11]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    #symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

# Generate some drum synthetic data similar to symbolic music

In [3]:
# lets make the data 400 in lenghth to match autoencoder imlementation 
#lets have 10 different sections each 40 in lenght -- ideal conditions 

import numpy as np

print("beat patter dictionary ")
## basic patters
downbeat    = np.array([1 if i % 4 == 0 else 0 for i in range(0,40)])
downbeat2x  = np.array([1 if i % 2 == 0 else 0 for i in range(0,40)])
high_hats   = np.array([2 if i % 4 == 1 else 0 for i in range(0,40)])
high_hats2x = np.array([2 if i % 2 == 1 else 0 for i in range(0,40)])
tom_drum    = np.array([3 if i % 4 == 2 else 0 for i in range(0,40)])

## combine basic patters
comb1 = downbeat   + high_hats 
comb2 = downbeat   + high_hats2x
comb3 = downbeat2x + high_hats2x
comb4 = downbeat   + high_hats2x  + tom_drum
comb5 = downbeat2x + high_hats


print(list(downbeat))
print(list(downbeat2x))
print(list(high_hats))
print(list(high_hats2x))
print(list(tom_drum), "\n")

print(list(comb1))
print(list(comb2))
print(list(comb3))
print(list(comb4))
print(list(comb5), "\n")


##here is the list we will permute
musical_sections = [downbeat, downbeat2x, high_hats, high_hats2x, tom_drum, comb1, comb2, comb3, comb4, comb5]

from itertools import permutations

synth_data = np.array(list(permutations(musical_sections))[0:10000]) ##okay but eventually you might need to trucate the permutations

#reshape and unravel
(a,b,c) = synth_data.shape
synth_data = synth_data.reshape((a, b*c))
synth_data = synth_data[:,None,:] #add channel dim
print("size of dataset ", synth_data.shape)

print("Example datapoint:\n ", synth_data[0])


beat patter dictionary 
[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0]
[0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2]
[0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0] 

[1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0]
[1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2]
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
[1, 2,

# Auto Econder

In [5]:
import torch
import torch.optim as optim
torch.manual_seed(42)

class autoencoder(torch.nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=40, padding=0, bias=False, stride=40)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=40, padding=0, bias=False, stride=40)
    def get_kernels(self):
        return self.encoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.encoder(x)
        return code
    def forward(self, x):
        code = self.encoder(x)
        reconstruct = self.decoder(code)
        return reconstruct


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)

model = autoencoder().to(device)
inputs = torch.tensor(synth_data).float().to(device)
out = model(inputs)

print("Input size: ", inputs.size())
print("Output size: ", out.size(), "\n")


loss_fn = torch.nn.L1Loss().to(device)
optimizer = optim.SGD(model.parameters(), lr=.05, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented

epochs = 1000
history = {"loss": []}
for i in range(epochs):
  optimizer.zero_grad()
  output = model(inputs)
  # print(output.size())
  loss = loss_fn(output, inputs)
  loss.backward()
  optimizer.step()
  history["loss"].append(float(loss))
  if i % 10 == 0: print("Epoch : {} \t Loss : {} \t ".format(i, round(float(loss),4)))




Using device:  cuda
Input size:  torch.Size([10000, 1, 400])
Output size:  torch.Size([10000, 1, 400]) 

Epoch : 0 	 Loss : 1.0373 	 
Epoch : 10 	 Loss : 0.97 	 
Epoch : 20 	 Loss : 0.9138 	 
Epoch : 30 	 Loss : 0.8538 	 
Epoch : 40 	 Loss : 0.7806 	 
Epoch : 50 	 Loss : 0.6896 	 
Epoch : 60 	 Loss : 0.6019 	 
Epoch : 70 	 Loss : 0.5341 	 
Epoch : 80 	 Loss : 0.494 	 
Epoch : 90 	 Loss : 0.4687 	 
Epoch : 100 	 Loss : 0.4452 	 
Epoch : 110 	 Loss : 0.4234 	 
Epoch : 120 	 Loss : 0.3982 	 
Epoch : 130 	 Loss : 0.3708 	 
Epoch : 140 	 Loss : 0.3396 	 
Epoch : 150 	 Loss : 0.3098 	 
Epoch : 160 	 Loss : 0.2836 	 
Epoch : 170 	 Loss : 0.2575 	 
Epoch : 180 	 Loss : 0.2405 	 
Epoch : 190 	 Loss : 0.2229 	 
Epoch : 200 	 Loss : 0.2061 	 
Epoch : 210 	 Loss : 0.1946 	 
Epoch : 220 	 Loss : 0.1807 	 
Epoch : 230 	 Loss : 0.1706 	 
Epoch : 240 	 Loss : 0.1571 	 
Epoch : 250 	 Loss : 0.1445 	 
Epoch : 260 	 Loss : 0.1301 	 
Epoch : 270 	 Loss : 0.118 	 
Epoch : 280 	 Loss : 0.1071 	 
Epoch : 290

In [6]:
#loss

#perfect plot
loss_plot = {"legend": "original", 
             "x": list(range(0,len(history["loss"]))), 
             "y": history["loss"]}
     

plot([loss_plot], "loss")

In [13]:
data = list(model.get_kernels()[4].tolist())

recunstruct = model(torch.tensor(inputs[0:1,:,:]).float())
# print(recunstruct.detach().numpy()[0,0,:])

#perfect plot
original_plot = {"legend": "original", 
                 "x": list(range(0,400)), 
                 "y": synth_data[0,0,:],
                 "marker_symbol": 'line-ne-open'}
                  
recunstruct_plot = {"legend": "reconstruct", 
             "x": list(range(0,400)), 
             "y": recunstruct.cpu().detach().numpy()[0,0,:],
             "marker_symbol": 'star'}      

plot([recunstruct_plot, original_plot], "Signal Comparison")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



# see kernels

In [21]:
kernel = model.get_kernels().cpu().numpy()[1].tolist()


print(kernels)


#perfect plot
kernels_plot  = {"legend": "original", 
                 "x": list(range(0,40)), 
                 "y": kernels,
                 "marker_symbol": 'star'}

plot([kernels_plot], "kernels_plot")

[0.11411722004413605, 0.2281462699174881, -0.1279941350221634, 0.08419380336999893, -0.04132836312055588, 0.1288684606552124, -0.16790996491909027, 0.03183377534151077, 0.13260750472545624, -0.018956424668431282, 0.04638779163360596, -0.03139796480536461, 0.11005119979381561, 0.11838158965110779, -0.01477130688726902, -0.08328737318515778, 0.1151202991604805, 0.12033765763044357, -0.16474699974060059, -0.020693162456154823, -0.0795137882232666, 0.07844676077365875, -0.1551574021577835, 0.043858110904693604, -0.13144798576831818, 0.024125559255480766, -0.13559570908546448, -0.1559942215681076, 0.008212151005864143, -0.05910422280430794, 0.051747098565101624, -0.19521725177764893, 0.11530069261789322, 0.12327869981527328, -0.14228510856628418, 0.036680132150650024, 0.01792593114078045, 0.2246626317501068, -0.07369596511125565, -0.11081349104642868]
